In [1]:
import pandas as pd
import utils 
import plotly.graph_objects as go


In [2]:
pair = "EUR_USD"
granularity = "H1"
ma_list =[8,16,32,64,128,256]

In [3]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-02-05T03:00:00.000000000Z,2151,1.07789,1.07810,1.07746,1.07790,1.07781,1.07802,1.07738,1.07782,1.07797,1.07819,1.07753,1.07798
1,2024-02-05T04:00:00.000000000Z,1450,1.07792,1.07800,1.07756,1.07790,1.07784,1.07793,1.07748,1.07783,1.07799,1.07807,1.07764,1.07798
2,2024-02-05T05:00:00.000000000Z,2683,1.07790,1.07823,1.07733,1.07822,1.07782,1.07816,1.07726,1.07814,1.07798,1.07830,1.07740,1.07829
3,2024-02-05T06:00:00.000000000Z,2469,1.07820,1.07849,1.07774,1.07780,1.07813,1.07841,1.07766,1.07772,1.07827,1.07857,1.07781,1.07787
4,2024-02-05T07:00:00.000000000Z,3854,1.07776,1.07837,1.07726,1.07803,1.07769,1.07829,1.07719,1.07796,1.07783,1.07845,1.07734,1.07810


In [5]:
df.describe()

,volume
count,3999.000000
mean,3147.517129
std,2345.575243
min,95.000000
25%,1557.500000
50%,2621.000000
75%,4082.500000
max,26382.000000


In [6]:
none_columns = ['time', 'volume']

In [7]:
mod_columns = [x for x in df.columns if x not in none_columns ]

In [8]:
df[mod_columns] = df[mod_columns].apply(pd.to_numeric)

In [9]:
df_ma = df[['time','mid_o','mid_h','mid_l','mid_c']].copy()

In [10]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2024-02-05T03:00:00.000000000Z,1.07789,1.07810,1.07746,1.07790
1,2024-02-05T04:00:00.000000000Z,1.07792,1.07800,1.07756,1.07790
2,2024-02-05T05:00:00.000000000Z,1.07790,1.07823,1.07733,1.07822
3,2024-02-05T06:00:00.000000000Z,1.07820,1.07849,1.07774,1.07780
4,2024-02-05T07:00:00.000000000Z,1.07776,1.07837,1.07726,1.07803


In [115]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [116]:
df_ma.dropna(inplace=True)

In [117]:
df_plot = df_ma.iloc[-100:].copy()

In [118]:
df_plot.shape


(100, 6)

In [119]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x= df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
     line=dict(width=1),opacity=1,
     increasing_fillcolor='#24A06B',
     decreasing_fillcolor='#CC2E3C',
     increasing_line_color='#2EC886',
     decreasing_line_color='#FF3A4C'))
     

fig.add_trace(go.Scatter(x=df_plot.time,
    y=df_plot.MA_8,
    line=dict(color="#027FC3",
    width=2),
    line_shape='spline',
    name='MA_8'))

fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10, color="#e1e1e1"),
    paper_bgcolor= "#1e1e1e",
    plot_bgcolor="#1e1e1e")

fig.update_xaxes(gridcolor="#1f292f",showgrid=True,fixedrange=True,rangeslider=dict(visible=False))
fig.update_yaxes(gridcolor="#1f292f",showgrid=True,fixedrange=True)

fig.show()